# AI Narrative Test Generator

This notebook generates a test to evaluate how accurately humans can identify dependencies within reasoning texts generated by reasoning models.

## Overview of the test

In each test question, participants are presented with:
- A problem and its reasoning steps generated by an AI model
- A **target step** (highlighted in yellow)
- Four **candidate steps** from earlier in the reasoning chain

**Task**: Participants must identify which candidate step, if removed, would cause the target step to change. There is exactly one correct answer per question.

**Hint**: Participants are shown what the target step will change to if the correct candidate step is removed.

## Requirements

- **API Access**: Enter your Together AI username in the `together_username` variable in the configuration cell below
- **Data Files**: Ensure alternative steps data and solution dataset CSV files are present in the working directory

## Configuration Notes

- This notebook supports test generation from multiple models (up to 2 simultaneously)
- Currently configured for single-model test generation using DeepSeek-R1
- To enable multi-model comparison, uncomment the `model_to_use2` lines throughout the notebook

In [ ]:
# Import required libraries
import json
import pandas as pd
import random
import os
from collections import defaultdict
import time
from tqdm import tqdm

# ==============================================================================
# CONFIGURATION PARAMETERS
# ==============================================================================

# Data source configuration
num_samples = 1  # Number of samples to process from the dataset
together_username = "insert_your_username_here"  # Replace with your Together AI username

# Model identifiers
my_qwen3 = together_username + "/Qwen/Qwen3-32B-0921e0c1"
my_r1 = together_username + "/deepseek-ai/DeepSeek-R1-d641e9ae"
r1 = "deepseek-ai/DeepSeek-R1"

r1_models = [r1]

# Active model selection
model_to_use1 = my_r1
# model_to_use2 = my_r1  # Uncomment to enable dual-model comparison

# Dataset configuration
gsm8k = "gsm8k"
dataset_to_use = gsm8k

# Test generation parameters
min_target_questions = 3  # Minimum number of questions to generate
show_alternative_step_box = True  # Display counterfactual step hints in HTML interface
prolific_mode = True  # Enable demographic data collection (for Prolific version)
select_questions_randomly = False  # False = balance answer position distribution; True = random selection

# Attention check configuration
attention_check_indices = [11, 21, 34]  # Question indices designated as attention checks

# Step filtering
max_target_step_limit = 19  # Maximum step index for target steps (None = no limit)

## Data Loading

This section loads the solution process data, including:
- Step-by-step reasoning chains generated by the AI model(s)
- Link relationships between steps (crucial/non-crucial dependencies)
- Main problem statements for each sample

In [17]:
def load_model_data(model_name, dataset_name, num_samples):
    """
    Load data for a specific model following the standard pattern.
    
    Args:
        model_name: The model identifier (e.g., 'my_qwen3', 'r1')
        dataset_name: The dataset identifier (e.g., 'gsm8k')
        num_samples: Number of samples to load
    
    Returns:
        List of sample data dictionaries with keys: sample_idx, links, solution_steps, main_question
    """
    # Load the alternative_steps_all data from the saved JSON file
    file_path = f'alternative_steps_all_{model_name.replace("/","")}_{dataset_name.replace("/","")}.json'
    
    with open(file_path, 'r') as f:
        links_between_steps_all = json.load(f)
    
    # Load the solution steps data (nodes_and_types_json) from the CSV file
    df = pd.read_csv(f"dataset_w_answers_{model_name.replace('/', '_')}_{dataset_name.replace('/', '_')}_{str(num_samples)}_samples.csv")
    
    # Convert the links data for each sample
    sample_data = []
    for sample_idx in range(len(df)):
        # Get the links for this sample (if available)
        if sample_idx < len(links_between_steps_all):
            links_for_sample = links_between_steps_all[sample_idx]
            
            # Parse the JSON in the nodes_and_types_json column for this sample
            solution_steps = json.loads(df['nodes_and_types_json'].iloc[sample_idx])
            
            # Extract the main question
            main_question = solution_steps[0]['text']
            
            sample_data.append({
                'sample_idx': sample_idx,
                'links': links_for_sample, # Keep original links with 'equivalent' as 'yes'/'no'/'unsure'
                'solution_steps': solution_steps,
                'main_question': main_question
            })
    
    return sample_data


# Load data for primary model
print(f"Loading data for primary model: {model_to_use1}")
sample_data = load_model_data(model_to_use1, dataset_to_use, num_samples)
print(f"Loaded {len(sample_data)} samples with link data for {model_to_use1}")  

# Load data for secondary model (optional - uncomment to enable)
# print(f"Loading data for secondary model: {model_to_use2}")
# sample_data_model2 = load_model_data(model_to_use2, dataset_to_use, 15)
# print(f"Loaded {len(sample_data_model2)} samples with link data for {model_to_use2}")

# Display summary statistics
print(f"\nData Loading Summary:")
print(f"- {model_to_use1}: {len(sample_data)} samples")
# print(f"- {model_to_use2}: {len(sample_data_model2)} samples")

Loading data for primary model: deepseek-ai/DeepSeek-R1
Loaded 1 samples with link data for deepseek-ai/DeepSeek-R1

Data Loading Summary:
- deepseek-ai/DeepSeek-R1: 1 samples


In [18]:
# ==============================================================================
# SAMPLE ORGANIZATION AND INTERLEAVING
# ==============================================================================
# This cell organizes samples from one or more models, interleaving them to
# ensure balanced representation in the generated test.

# Extract sample indices from loaded data
sample_indexes1 = [sample['sample_idx'] for sample in sample_data]
# sample_indexes2 = [sample['sample_idx'] for sample in sample_data_model2]  # For dual-model mode

# Initialize storage for interleaved samples
interleaved_samples = []

# Create lookup dictionaries indexed by sample_idx for efficient access
samples_by_idx1 = {sample['sample_idx']: sample for sample in sample_data}
# samples_by_idx2 = {sample['sample_idx']: sample for sample in sample_data_model2}  # For dual-model mode

# Get all unique sample indices and sort for consistent ordering
# all_sample_indexes = sorted(set(sample_indexes1 + sample_indexes2))  # For dual-model mode
all_sample_indexes = sorted(set(sample_indexes1))

# Annotate each sample with its source model name
for sample in samples_by_idx1.values():
    sample['model_name'] = model_to_use1
# for sample in samples_by_idx2.values():  # For dual-model mode
#     sample['model_name'] = model_to_use2

# Interleave samples: alternate between models for balanced representation
for sample_idx in all_sample_indexes:
    # Add sample from primary model if available
    if sample_idx in samples_by_idx1:
        interleaved_samples.append(samples_by_idx1[sample_idx])
    
    # Add sample from secondary model if available (dual-model mode)
    # if sample_idx in samples_by_idx2:
    #     interleaved_samples.append(samples_by_idx2[sample_idx])

# Create model-indexed dictionary for later reference
# samples_by_model = {model_to_use1: samples_by_idx1, model_to_use2: samples_by_idx2}  # For dual-model mode
samples_by_model = {model_to_use1: samples_by_idx1}

## Question Generation

This section processes the step dependency data to generate test questions. The process involves:

1. **Phase 1**: Generate all valid question candidates from the sample data
2. **Phase 2**: Select an optimal subset that balances answer position distribution and model/sample representation

Each question presents four prior steps, where exactly one is crucial to the target step.

In [19]:
# ==============================================================================
# PHASE 1: QUESTION CANDIDATE GENERATION
# ==============================================================================
"""
This section generates all valid question candidates from the sample data.

Strategy:
- If select_questions_randomly=False: Optimize for balanced answer position distribution
  across questions (positions A, B, C, D)
- If select_questions_randomly=True: Randomly select questions for even sample coverage

Each candidate must have:
- Exactly 1 crucial step (correct answer)
- At least 3 non-crucial steps (incorrect options)
- Target step within max_target_step_limit (if specified)
"""

sample_data_list = interleaved_samples
min_questions_target = min_target_questions

all_generated_candidates = []
# Track answer position distribution during generation (indices 0-3 correspond to A-D)
position_counts_for_generation_phase = [0, 0, 0, 0]

# Generate all possible question candidates
for sample_info in sample_data_list:
    sample_idx = sample_info['sample_idx']
    links_for_sample = sample_info['links']
    model_name = sample_info['model_name']
    solution_steps_for_sample = sample_info.get('solution_steps', [])
    if not isinstance(links_for_sample, list) or not isinstance(solution_steps_for_sample, list):
        print(f"Skipping sample {sample_idx} due to invalid links or solution_steps format.")
        continue

    target_to_links_for_sample = defaultdict(list)
    for link in links_for_sample:
        target_idx = link['target_step_index']
        target_to_links_for_sample[target_idx].append(link)

    for target_idx_original, links_for_this_target in target_to_links_for_sample.items():
        # Add check for max_target_step_limit
        if max_target_step_limit is not None and target_idx_original > max_target_step_limit:
            continue # Skip targets that are too late in the solution steps

        crucial_steps_with_alt = []
        non_crucial_steps_yes = []
        for link in links_for_this_target:
            prior_step_idx = link['omitted_step_index']
            if prior_step_idx >= len(solution_steps_for_sample): continue
            prior_step_text = solution_steps_for_sample[prior_step_idx]['text']
            step_info = {'step_index': prior_step_idx, 'text': prior_step_text, 'equivalent': link['equivalent']}
            if link['equivalent'] == 'no':
                step_info['counterfactual_text'] = link.get('alternative_step', '')
                crucial_steps_with_alt.append(step_info)
            elif link['equivalent'] == 'yes':
                non_crucial_steps_yes.append(step_info)

        if not crucial_steps_with_alt:
            continue
        if len(non_crucial_steps_yes) < 3: # Must have at least 3 incorrect options
            continue
        
        correct_choice_candidate = None
        selected_incorrect_choices = []

        if not select_questions_randomly:
            possible_configurations_for_this_target = []

            for current_potential_correct_choice in crucial_steps_with_alt:
                non_crucial_before = sorted([s for s in non_crucial_steps_yes if s['step_index'] < current_potential_correct_choice['step_index']], key=lambda x: x['step_index'])
                non_crucial_after = sorted([s for s in non_crucial_steps_yes if s['step_index'] > current_potential_correct_choice['step_index']], key=lambda x: x['step_index'])

                feasible_strategies_for_this_candidate = []
                # Check position 0 (Correct is 1st: 0 before, 3 after)
                if len(non_crucial_after) >= 3:
                    feasible_strategies_for_this_candidate.append({'pos': 0, 'before_needed': 0, 'after_needed': 3})
                # Check position 1 (Correct is 2nd: 1 before, 2 after)
                if len(non_crucial_before) >= 1 and len(non_crucial_after) >= 2:
                    feasible_strategies_for_this_candidate.append({'pos': 1, 'before_needed': 1, 'after_needed': 2})
                # Check position 2 (Correct is 3rd: 2 before, 1 after)
                if len(non_crucial_before) >= 2 and len(non_crucial_after) >= 1:
                    feasible_strategies_for_this_candidate.append({'pos': 2, 'before_needed': 2, 'after_needed': 1})
                # Check position 3 (Correct is 4th: 3 before, 0 after)
                if len(non_crucial_before) >= 3:
                    feasible_strategies_for_this_candidate.append({'pos': 3, 'before_needed': 3, 'after_needed': 0})

                for strategy in feasible_strategies_for_this_candidate:
                    score = position_counts_for_generation_phase[strategy['pos']]
                    possible_configurations_for_this_target.append({
                        'score': score,
                        'correct_choice_candidate': current_potential_correct_choice,
                        'strategy': strategy,
                        'non_crucial_before': non_crucial_before,
                        'non_crucial_after': non_crucial_after
                    })
            
            if possible_configurations_for_this_target:
                possible_configurations_for_this_target.sort(key=lambda x: (x['score'], x['strategy']['pos']))
                best_overall_configuration = possible_configurations_for_this_target[0]
                
                correct_choice_candidate = best_overall_configuration['correct_choice_candidate']
                best_strategy = best_overall_configuration['strategy']
                non_crucial_before_for_best = best_overall_configuration['non_crucial_before']
                non_crucial_after_for_best = best_overall_configuration['non_crucial_after']
                
                temp_selected_incorrect = []
                if best_strategy['before_needed'] > 0:
                    temp_selected_incorrect.extend(random.sample(non_crucial_before_for_best, best_strategy['before_needed']))
                if best_strategy['after_needed'] > 0:
                    temp_selected_incorrect.extend(random.sample(non_crucial_after_for_best, best_strategy['after_needed']))
                
                if len(temp_selected_incorrect) == 3:
                    selected_incorrect_choices = temp_selected_incorrect

            # Fallback if the above didn't yield 3 choices or no configurations found
            if len(selected_incorrect_choices) != 3:
                if not correct_choice_candidate: # If no best_overall_configuration was found/used
                    if crucial_steps_with_alt: # Ensure crucial_steps_with_alt is not empty
                         correct_choice_candidate = random.choice(crucial_steps_with_alt)
                    else:
                        continue # Should have been caught earlier, but as a safeguard
                
                # Reset selected_incorrect_choices for random sampling if fallback is needed
                selected_incorrect_choices = [] 
                if len(non_crucial_steps_yes) >= 3:
                    selected_incorrect_choices = random.sample(non_crucial_steps_yes, 3)
                # If still not 3, the final check will catch it

        else: # select_questions_randomly is True
            if crucial_steps_with_alt: # Ensure crucial_steps_with_alt is not empty
                correct_choice_candidate = random.choice(crucial_steps_with_alt)
            else:
                continue # No crucial steps to choose from
            if len(non_crucial_steps_yes) >= 3:
                selected_incorrect_choices = random.sample(non_crucial_steps_yes, 3)
            # If not 3 incorrect choices, final check will skip

        # Ensure we have a correct choice and exactly 3 incorrect choices, otherwise skip this candidate
        if not correct_choice_candidate or len(selected_incorrect_choices) != 3:
            continue

        correct_choice_data = {'step_index': correct_choice_candidate['step_index'], 'text': correct_choice_candidate['text'], 'is_crucial': True}
        incorrect_choices_data = [{'step_index': npc['step_index'], 'text': npc['text'], 'is_crucial': False} for npc in selected_incorrect_choices]
        choices_data = [correct_choice_data] + incorrect_choices_data

        # sort choices_data by step_index
        choices_data.sort(key=lambda x: x['step_index'])

        if target_idx_original >= len(solution_steps_for_sample): continue
        target_text = solution_steps_for_sample[target_idx_original]['text']

        final_choices_for_q = []
        answer_indices_for_q = []
        for i, choice_d in enumerate(choices_data):
            final_choices_for_q.append({'step_index': choice_d['step_index'], 'text': choice_d['text'], 'is_crucial': choice_d['is_crucial']})
            if choice_d['is_crucial']: answer_indices_for_q.append(i)

        # Determine the achieved HTML position based on the choices - it is the relative position of the correct choice
        achieved_html_pos = next((i for i, choice in enumerate(final_choices_for_q) if choice['is_crucial']), None)


        question_data_dict = {
            'question_number': 0, # Will be renumbered later
            'target_step_index': target_idx_original,
            'target_step_text': target_text,
            'counterfactual_starting_text': correct_choice_candidate['counterfactual_text'],
            'choices': final_choices_for_q,
            'answer_indices': [achieved_html_pos],
            'sample_idx': sample_idx, # Store sample_idx with question data
            "achieved_html_pos": achieved_html_pos,
            'model_name': model_name
        }
        all_generated_candidates.append({
            'question_data': question_data_dict,
            'achieved_html_pos': achieved_html_pos,
            'sample_idx': sample_idx,
            'target_idx_original': target_idx_original,
            
        })
        if achieved_html_pos is not None and 0 <= achieved_html_pos < len(position_counts_for_generation_phase):
             position_counts_for_generation_phase[achieved_html_pos] += 1

# Display generation phase statistics
print(f"\nPhase 1 Complete: Generated {len(all_generated_candidates)} question candidates")
print(f"Answer position distribution (A, B, C, D): {position_counts_for_generation_phase}")



Phase 1 Complete: Generated 2 question candidates
Answer position distribution (A, B, C, D): [1, 0, 0, 1]


In [20]:
# ==============================================================================
# PHASE 2: OPTIMAL QUESTION SELECTION
# ==============================================================================
"""
Select the final set of questions using a greedy balancing algorithm.

Balancing priorities (in order):
1. Answer position distribution (ensure roughly equal A, B, C, D answers)
2. Questions per model (for multi-model tests)
3. Questions per sample (avoid over-sampling any single problem)

The algorithm iteratively selects the candidate that best improves balance
across these dimensions.
"""

final_selected_question_data_list = []
final_html_position_counts = [0, 0, 0, 0]  # Counts for positions 0, 1, 2, 3
questions_selected_per_model = defaultdict(int)
questions_selected_per_sample = defaultdict(int)

if not all_generated_candidates:
    raise ValueError("No valid question candidates generated. Check the input data and links format.")
else:
    # Make a mutable copy of the candidates to select from
    remaining_candidates = list(all_generated_candidates)

    for _ in range(min_questions_target): # Iterate up to min_questions_target times
        if not remaining_candidates:
            print(f"Ran out of candidates after selecting {len(final_selected_question_data_list)} questions.")
            break

        scored_candidates = []
        for idx, candidate_info in enumerate(remaining_candidates):
            q_data = candidate_info['question_data']
            achieved_pos = candidate_info['achieved_html_pos']
            
            # Ensure achieved_pos is valid to prevent IndexError
            if achieved_pos is None or not (0 <= achieved_pos < len(final_html_position_counts)):
                # print(f"Warning: Candidate with invalid achieved_html_pos ({achieved_pos}) skipped.")
                continue # Skip this candidate if position is invalid

            score_html = final_html_position_counts[achieved_pos]
            score_model = questions_selected_per_model[q_data['model_name']]
            score_sample = questions_selected_per_sample[q_data['sample_idx']]
            
            # Add the original index from remaining_candidates to break ties consistently
            # and to allow easy removal later.
            scored_candidates.append(
                (score_html, score_model, score_sample, idx, candidate_info)
            )

        if not scored_candidates: # All remaining candidates might have invalid pos or been filtered
            print("No more valid candidates to score in this iteration.")
            break

        # Sort by the scores: lower is better for all criteria. original_idx_in_remaining for stable sort.
        scored_candidates.sort(key=lambda x: (x[1], x[0], x[2], x[3]))

        best_scored_candidate_tuple = scored_candidates[0]
        original_idx_in_remaining = best_scored_candidate_tuple[3] # This is the index in the *current* remaining_candidates list
        best_candidate_info = best_scored_candidate_tuple[4]
        
        final_selected_question_data_list.append(best_candidate_info['question_data'])
        
        # Update counts based on the selected candidate
        final_html_position_counts[best_candidate_info['achieved_html_pos']] += 1
        questions_selected_per_model[best_candidate_info['question_data']['model_name']] += 1
        questions_selected_per_sample[best_candidate_info['question_data']['sample_idx']] += 1
        
        # Remove the selected candidate from remaining_candidates using its original index from that list
        remaining_candidates.pop(original_idx_in_remaining)

# Renumber final questions (order is now determined by the greedy selection)
for i, q_data in enumerate(final_selected_question_data_list):
    q_data['question_number'] = i + 1

all_test_questions = final_selected_question_data_list
html_position_counts = final_html_position_counts # Used by later cells

# Display final selection statistics
print(f"\n{'='*70}")
print(f"Phase 2 Complete: Question Selection Summary")
print(f"{'='*70}")
print(f"Total questions generated: {len(all_test_questions)}")
print(f"Target question count: {min_questions_target}")
print(f"\nAnswer position distribution (A, B, C, D): {html_position_counts}")
print(f"Questions per model: {dict(questions_selected_per_model)}")
# print(f"Questions per sample: {dict(questions_selected_per_sample)}")  # Verbose output - uncomment if needed

print(f"\nQuestion ordering (as determined by selection algorithm):")
for i, q in enumerate(all_test_questions):
    print(f"  Q{q['question_number']}: Answer Position={chr(65 + q.get('achieved_html_pos', 0))}, "
          f"Sample={q['sample_idx']}, Model={q['model_name'][:25]}, Target Step={q['target_step_index']}")

Ran out of candidates after selecting 2 questions.

Phase 2 Complete: Question Selection Summary
Total questions generated: 2
Target question count: 3

Answer position distribution (A, B, C, D): [1, 0, 0, 1]
Questions per model: {'deepseek-ai/DeepSeek-R1': 2}

Question ordering (as determined by selection algorithm):
  Q1: Answer Position=A, Sample=0, Model=deepseek-ai/DeepSeek-R1, Target Step=9
  Q2: Answer Position=D, Sample=0, Model=deepseek-ai/DeepSeek-R1, Target Step=11


In [21]:
# ==============================================================================
# QUESTION REORDERING
# ==============================================================================
# Group questions by sample and model, then order by target step within each group
# This creates a logical flow where questions about the same problem appear consecutively

all_test_questions.sort(key=lambda q: (q['sample_idx'], q['model_name'], q['target_step_index']))

# Renumber questions sequentially after reordering
for i, q in enumerate(all_test_questions):
    q['question_number'] = i + 1

In [22]:
# ==============================================================================
# FINAL DISTRIBUTION ANALYSIS
# ==============================================================================
# Verify the distribution of questions across models and samples

# Count questions per model
model_question_counts = defaultdict(int)
for q in all_test_questions:
    model_question_counts[q['model_name']] += 1

print(f"\nQuestion distribution by model:")
for model_name, count in model_question_counts.items():
    print(f"  {model_name}: {count} questions")

# Count questions per sample
sample_question_counts = defaultdict(int)
for q in all_test_questions:
    sample_question_counts[q['sample_idx']] += 1

print(f"\nQuestion distribution by sample:")
for sample_idx, count in sorted(sample_question_counts.items()):
    print(f"  Sample {sample_idx}: {count} questions")


Question distribution by model:
  deepseek-ai/DeepSeek-R1: 2 questions

Question distribution by sample:
  Sample 0: 2 questions


In [23]:
# ==============================================================================
# SAVE TEST QUESTIONS
# ==============================================================================
# Create a timestamped directory and save the raw question data

time_stamp = time.strftime("%Y%m%d-%H%M%S")
directory_name = f"tests/{model_to_use1.replace('/','_')}_{dataset_to_use.replace('/','_')}_{str(num_samples)}_samples_{time_stamp}"

# Create output directory
os.makedirs(directory_name, exist_ok=True)

# Save raw question data
with open(f"{directory_name}/test_questions.json", 'w') as f:
    json.dump(all_test_questions, f, indent=4)

print(f"Test questions saved to: {directory_name}/test_questions.json")
    

Test questions saved to: tests/deepseek-ai_DeepSeek-R1_gsm8k_1_samples_20251117-172503/test_questions.json


## Test Formatting and Export

This section formats the test questions for display and exports them in multiple formats:
- Structured JSON for programmatic access
- Interactive HTML for participant administration

In [24]:
def format_test_for_display(questions, samples, show_alt_box_toggle):
    """
    Format the test questions for display or export.
    Now supports questions from multiple samples and toggles alternative box visibility.
    """
    instruction_text = (
        "You are presented with a problem and a chain of thought generated by an AI when solving it. For each question, the chain of thought is shown up to a 'target step' (highlighted in yellow).\n\n"
        "<strong>Your task:</strong> From the four options provided, select the step that is crucial for the target step. All other steps being equal, the crucial step absence would cause the AI to generate a significantly different target step. \n\n"
        "Only one of the four options is crucial. The other three, if absent, would not significantly change the target step."
    )
    if show_alt_box_toggle:
        instruction_text += (
            "\n\nA snippet of text below the target step shows how the target step in yellow will begin if the crucial step was indeed omitted. Use this as a hint."
        )

    test_format = {
        'title': 'Step Dependency Questionnaire',
        'instructions': instruction_text,
        'samples': samples,  # Store all sample data for reference
        'questions': []
    }
    
    for q in questions:
        # Get the main question from the appropriate sample
        sample_idx = q['sample_idx']
        model_name = q['model_name']
        main_question = samples[model_name][sample_idx]['main_question']
        
        formatted_q = {
            'question_number': q['question_number'],
            'sample_idx': sample_idx,
            'main_question': main_question,
            'target_step_index': q['target_step_index'],
            'target_step_text': q['target_step_text'],
            'counterfactual_starting_text': q.get('counterfactual_starting_text', '') if show_alt_box_toggle else '',
            'choices': [
                {
                    'choice_letter': chr(65 + i),  # A, B, C, D
                    'step_index': c['step_index'],
                    'text': c['text']
                } for i, c in enumerate(q['choices'])
            ],
            'correct_answers': [chr(65 + i) for i in q['answer_indices']],  # A, B, C, D (will be single char)
            'model_name': q['model_name'],
        }
        test_format['questions'].append(formatted_q)
    
    return test_format

# Format the test with display-ready structure
formatted_test = format_test_for_display(all_test_questions, samples_by_model, show_alternative_step_box)

# Save formatted test as JSON
with open(f'{directory_name}/parent_step_detection_test.json', 'w') as f:
    json.dump(formatted_test, f, indent=2)

print(f"Formatted test saved to: {directory_name}/parent_step_detection_test.json")

Formatted test saved to: tests/deepseek-ai_DeepSeek-R1_gsm8k_1_samples_20251117-172503/parent_step_detection_test.json


In [25]:
# Save a second copy of the formatted test for convenience
with open(f"{directory_name}/formatted_test.json", 'w') as f:
    json.dump(formatted_test, f, indent=4)

print(f"Additional formatted copy saved to: {directory_name}/formatted_test.json")


Additional formatted copy saved to: tests/deepseek-ai_DeepSeek-R1_gsm8k_1_samples_20251117-172503/formatted_test.json


## HTML Test Generation

This section generates an interactive HTML interface for test administration.

**Features:**
- Responsive web interface with step-by-step navigation
- Visual highlighting of target steps and answer choices
- Optional counterfactual hints (controlled by `show_alternative_step_box`)
- Demographic data collection (when `prolific_mode` is enabled)
- Time tracking per question and overall
- Question randomization while preserving sample grouping
- Integration with Google Apps Script for data submission

In [26]:
def generate_html_test(formatted_test, show_alt_box_toggle, prolific_mode=False):
    """
    Generate a complete HTML interface for test administration.
    
    Args:
        formatted_test (dict): Structured test data with questions and metadata
        show_alt_box_toggle (bool): Whether to display counterfactual step hints
        prolific_mode (bool): Enable demographic collection and continuous submission
    
    Returns:
        str: Complete HTML document with embedded JavaScript
    
    Features:
        - Progressive display of reasoning steps up to target
        - Interactive step selection with visual feedback
        - Yellow highlighting of target steps
        - Optional counterfactual hints (if show_alt_box_toggle=True)
        - Question navigation with answer validation
        - Correct answer display (disabled in prolific_mode)
        - Answer position balancing (A, B, C, D)
        - Multi-sample support with proper attribution
        - Google Apps Script integration for data submission
        - Demographic form with validation (when prolific_mode=True)
        - Incremental progress submission (when prolific_mode=True)
        - Per-question and total time tracking
        - Sample-grouped question randomization
    """
    html = f"""<!DOCTYPE html>
<html>
<head>
    <title>{formatted_test['title']}</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 900px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #2c3e50; text-align: center; }}
        .instructions {{ background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 5px; white-space: pre-line; }}
        .instructions h2 {{ margin-bottom: 0.5rem; }}
        .instructions p {{ margin-top: 0.25rem; }}
        .main-problem {{ background-color: #e9f7ef; padding: 15px; border-radius: 5px; margin-bottom: 20px; }}
        .solution-process {{ margin-bottom: 30px; }}
        .step-container {{ display: flex; align-items: center; margin-bottom: 10px; }}
        .instruction-area {{ width: 180px; margin-right: 15px; flex-shrink: 0; font-style: italic; color: #555; }}
        .step-box {{ flex-grow: 1; }}
        .step {{ background-color: #f5f5f5; padding: 15px; border-radius: 5px; }}
        .step.choice {{ cursor: pointer; transition: all 0.2s; }}
        .step.choice:hover {{ background-color: #e9ecef; }}
        .step.selected {{ background-color: #d4edda; border: 2px solid #28a745; }}
        .target-step {{ background-color: #fff3cd; border: 2px solid #ffc107; padding: 20px; font-size: 1.1em; }}
        .counterfactual-text-container {{ display: flex; align-items: flex-start; margin-bottom: 10px; }} /* For counterfactual text alignment */
        .counterfactual-text-info {{ background-color: #e8f0fe; border-left: 4px solid #5b9bd5; padding: 10px; border-radius: 5px; margin-top: 5px; margin-bottom:10px; font-style: italic; }}
        .step.correct {{ background-color: #d4edda; border-color: #28a745; }}
        .step.incorrect {{ background-color: #f8d7da; border-color: #dc3545; }}
        .step-index {{ font-weight: bold; color: #3498db; }}
        .controls {{ display: flex; justify-content: flex-end; margin-top: 20px; }}
        .controls .left-buttons {{ display: flex; gap: 10px; margin-right: auto; }}
        button {{ background-color: #3498db; color: white; border: none; padding: 10px 15px; border-radius: 5px; cursor: pointer; }}
        button:hover {{ background-color: #2980b9; }}
        button:disabled {{ background-color: #cccccc; cursor: not-allowed; }}
        .hidden {{ display: none !important; }} /* Added !important to ensure override */
        .result {{ margin-top: 20px; padding: 15px; border-radius: 5px; }}
        .progress {{ text-align: center; margin-bottom: 15px; font-weight: bold; }}
        .choice-indicator {{ color: #3498db; font-weight: bold; margin-right: 5px; }}
        .badge {{ display: inline-block; padding: 3px 8px; border-radius: 12px; color: white; font-size: 0.8em; margin-left: 5px; }}
        .badge-crucial {{ background-color: #28a745; }}
        .badge-not-crucial {{ background-color: #dc3545; }}
        .sample-info {{ background-color: #eef; padding: 8px; font-style: italic; margin-bottom: 15px; border-radius: 3px; }}
        .submit-container {{ text-align: center; margin-top: 30px; padding-top: 20px; border-top: 1px solid #ccc; }}
        
        /* Additional styles for demographic form */
        .demographic-form {{ background-color: #f8f9fa; padding: 25px; border-radius: 8px; margin-bottom: 30px; }}
        .form-group {{ margin-bottom: 20px; }}
        .form-group label {{ display: block; margin-bottom: 5px; font-weight: bold; }}
        .form-group .radio-label {{ display: flex; flex-direction: column; align-items: flex-start; font-weight: normal; margin-bottom: 3px; }}
        .form-group .radio-label input[type="radio"] {{
            margin-right: 8px;
            margin-top: 10px;
            order: 2;
            width: 10px;
        }}

        .form-group .radio-label span {{
            order: 2;
            margin-top: -15px;
            margin-left: 25px;
        }}
        .form-group .radio-label span {{ order: 2; margin-top: -15px; }}
        .form-group input, .form-group select {{ width: 100%; padding: 8px; border-radius: 4px; border: 1px solid #ddd; box-sizing: border-box; }}
        .form-group .field-description {{ color: #666; font-size: 0.9em; margin-top: 3px; }}
        .form-group .field-explanation {{ background-color: #eef; padding: 10px; border-radius: 4px; margin-top: 5px; font-size: 0.9em; border: 1px solid #dde; }}
        .required {{ color: #d9534f; }}
        .error-message {{ color: #d9534f; margin-top: 5px; font-size: 0.9em; display: none; }}
        .separator {{ border-top: 2px solid #ccc; margin: 20px 0; }}
        .intro-ct {{ font-size: 1.2em; background-color: #eef; padding: 10px; border-radius: 5px; margin-bottom: 20px; }}
        .quote {{ font-family: 'Georgia', serif; font-style: italic; margin-bottom: 10px; font-size: calc(1em + 8px); }}
    </style>
</head>
<body>
    <h1>{formatted_test['title']}</h1>
"""
    
    # Add the demographic form if prolific mode is enabled
    if prolific_mode:
        html += """
    <div id="demographic-container" class="demographic-form">
        <h2>Participant Information</h2>
        <p>Please fill in the following information before starting the test:</p>
        <div class="form-group">
            <label for="prolific-id">Prolific ID <span class="required">*</span></label>
            <input type="text" id="prolific-id" name="prolific-id" required>
            <div class="field-description">Your Prolific ID can be found in your Prolific account settings</div>
            <div id="prolific-id-error" class="error-message">Prolific ID is required</div>
        </div>
        <div class="separator"></div>
        <div class="form-group">
            <label for="education">Education Level <span class="required">*</span></label>
            <select id="education" name="education" required onchange="handleEducationChange()">
                <option value="">-- Select your highest education level --</option>
                <option value="high-school">High school graduate</option>
                <option value="bachelors">Bachelor\'s degree</option>
                <option value="masters">Master\'s degree</option>
                <option value="doctorate">Doctorate degree</option>
            </select>
            <div id="education-error" class="error-message">Education level is required</div>
        </div>
        <div class="separator"></div>
        <div class="form-group hidden" id="bachelor-degree-field-container">
            <label for="bachelor-degree-field">What was your Bachelor\'s degree in?</label>
            <input type="text" id="bachelor-degree-field" name="bachelor-degree-field">
            <div class="field-description">If you do not have a Bachelor\'s degree, or if this question is not applicable based on your education level, please leave this field empty.</div>
        </div>
        <div class="separator"></div>
        <div class="form-group">
            <label for="age">Age <span class="required">*</span></label>
            <select id="age" name="age" required>
                <option value="">-- Select your age range --</option>
                <option value="18-24">18-24</option>
                <option value="25-34">25-34</option>
                <option value="35-44">35-44</option>
                <option value="45-54">45-54</option>
                <option value="55-64">55-64</option>
                <option value="65+">65+</option>
                <option value="prefer-not-to-say">Prefer not to say</option>
            </select>
            <div id="age-error" class="error-message">Age range is required</div>
        </div>
        <div class="separator"></div>
        <div class="form-group">
            <label for="pronouns">Preferred pronouns <span class="required">*</span></label>
            <select id="pronouns" name="pronouns" required>
                <option value="">-- Select your pronouns --</option>
                <option value="he">He\Him</option>
                <option value="she">She\Her</option>
                <option value="other">Other\prefer not to say</option>
            </select>
            <div id="pronouns-error" class="error-message">Pronouns selection is required</div>
        </div>
        <div class="separator"></div>
        <div class="form-group">
            <label for="frequency">How often do you use AI systems that you interact with through text? <span class="required">*</span></label>
            <select id="frequency" name="frequency" required>
                <option value="">-- Select usage frequency --</option>
                <option value="once-month-or-less">Once a month or less</option>
                <option value="once-week">Once a week</option>
                <option value="few-times-week">A few times a week</option>
                <option value="once-day">Once a day</option>
                <option value="multiple-times-day">Multiple times a day</option>
            </select>
            <div id="frequency-error" class="error-message">Usage frequency is required</div>
        </div>
        <div class="separator"></div>
        <div class="intro-ct">New AI feature: Some AI systems provide a text described as a "chain-of-thought" (also called "thinking" or "reasoning") text that the AI generates before producing the final answer.</div>
        <div class="separator"></div>
        <div class="form-group">
            <label for="familiar-feature">Were you familiar with this AI feature? <span class="required">*</span></label>
            <div class="radio-label">
                <input type="radio" id="familiar-feature-yes" name="familiar-feature" value="yes" required>
                <span>Yes</span>
            </div>
            <div class="radio-label">
                <input type="radio" id="familiar-feature-no" name="familiar-feature" value="no">
                <span>No</span>
            </div>
            <div id="familiar-feature-error" class="error-message">Please indicate your familiarity</div>
        </div>
        <div class="separator"></div>
        <p>Please indicate how much you agree or disagree with each of the following statements by selecting one option per statement.</p>
        <div class="form-group">
            <p class="quote">"Reading the chain-of-thought text makes the model's process more transparent by reflecting which reasoning steps it actually performed and how each step leads to the next."</p>
            <label class="radio-label"><input type="radio" name="likert1" value="5"><span>Strongly agree</span></label>
            <label class="radio-label"><input type="radio" name="likert1" value="4"><span>Agree</span></label>
            <label class="radio-label"><input type="radio" name="likert1" value="3"><span>Neutral</span></label>
            <label class="radio-label"><input type="radio" name="likert1" value="2"><span>Disagree</span></label>
            <label class="radio-label"><input type="radio" name="likert1" value="1" required><span>Strongly disagree</span></label>
        </div>
        <div class="form-group">
            <p class="quote">"Being able to inspect an AI's chain-of-thought text will increase my trust in its answers."</p>
            <label class="radio-label"><input type="radio" name="likert2" value="5"><span>Strongly agree</span></label>
            <label class="radio-label"><input type="radio" name="likert2" value="4"><span>Agree</span></label>
            <label class="radio-label"><input type="radio" name="likert2" value="3"><span>Neutral</span></label>
            <label class="radio-label"><input type="radio" name="likert2" value="2"><span>Disagree</span></label>
            <label class="radio-label"><input type="radio" name="likert2" value="1" required><span>Strongly disagree</span></label>
        </div>
        <div class="form-group">
            <p class="quote">"I expect to perform well in a task where, given an AI chain-of-thought for a simple grade-school math problem, I must choose one of four earlier steps whose removal would change a specified later step."</p>
            <label class="radio-label"><input type="radio" name="likert3" value="5"><span>Strongly agree</span></label>
            <label class="radio-label"><input type="radio" name="likert3" value="4"><span>Agree</span></label>
            <label class="radio-label"><input type="radio" name="likert3" value="3"><span>Neutral</span></label>
            <label class="radio-label"><input type="radio" name="likert3" value="2"><span>Disagree</span></label>
            <label class="radio-label"><input type="radio" name="likert3" value="1" required><span>Strongly disagree</span></label>
            
        </div>

        <div class="controls">
            <button onclick="startTest()">Start Test</button>
        </div>
    </div>
    <div id="test-container" class="hidden">"""
    else:  # If not in prolific mode, make the test visible immediately
        html += """
    <div id="test-container">
"""
    
    # Add instructions and progress indicator
    html += f"""
        <div class="instructions">
            <h2>Instructions</h2>
            <p>{formatted_test['instructions'].replace('**Your task:**', '<strong>Your task:</strong>')}</p>
        </div>
        
        <div class="progress">
            <span>Question: <span id="current-question">1</span>/<span id="total-questions">{len(formatted_test['questions'])}</span></span>
        </div>
"""
    
    # Generate HTML for each question
    for i, q in enumerate(formatted_test['questions']):
        # Skip empty questions
        if not q:
            continue
            
        display_style = 'block' if i == 0 else 'none'  # First question visible, others hidden initially
        
        # Get the sample index and main question for this question
        sample_idx = q['sample_idx']
        main_question = q['main_question']
        model_name = q['model_name']
        # Get solution steps for this specific sample
        solution_steps = samples_by_model[model_name][sample_idx]['solution_steps']
        
        # get the index of model used
        model_index = 2 - list(samples_by_model.keys()).index(model_name)

        html += f"""
        <div id="question-container-{i+1}" class="question-container" style="display: {display_style};" data-original-index="{i+1}" data-sample-idx="{sample_idx}" data-model-name="{model_name}">
            <div class="main-problem">
                <h2>Problem</h2>
                <p>{main_question}</p>
            </div>
                  
            <div class="solution-process">
                <h3>AI Chain-of-thought (up to Target Step)</h3>
"""
        
        # Get the target step index
        target_idx = q['target_step_index']
        counterfactual_text = q.get('counterfactual_starting_text', '')
        
        # Sort choices by step_index to ensure letters match placement
        #sorted_choices = sorted(q['choices'], key=lambda x: x['step_index'])
        sorted_choices = q['choices']
        # Recreate the choice_letters map with sorted choices
        choice_letters = {}
        for idx, choice_item in enumerate(sorted_choices): # Renamed choice to choice_item to avoid conflict
            letter = chr(65 + idx)  # A, B, C, D in order
            choice_letters[choice_item['step_index']] = letter
        
        # Get correct answers for this question (need to remap since we sorted choices)
        correct_indices = []
        for c_idx, choice_item in enumerate(sorted_choices): # Renamed choice to choice_item
            if any(orig_choice['step_index'] == choice_item['step_index'] and 
                   orig_choice['choice_letter'] in q['correct_answers'] 
                   for orig_choice in q['choices']):
                correct_indices.append(c_idx)
        
        correct_answers_list = [chr(65 + idx) for idx in correct_indices] # Will be a single char array
        correct_answers_json = ', '.join([f"'{a}'" for a in correct_answers_list])
        
        # Display steps from solution_steps (only up to and including target_idx)

        for step_idx, step in enumerate(solution_steps):
            # Skip the first step (index 0) as it's the main question
            if step_idx == 0:
                continue
                
            # Only display steps up to and including the target step
            if step_idx > target_idx:
                continue
                
            # Get the text of the step
            step_text = step['text']
            
            # Start a consistent flex container for all step types
            html += '<div class="step-container">'
            
            # Left column: instruction text (or empty space for consistency)
            html += '<div class="instruction-area">'
            if step_idx in choice_letters:
                # Only show instruction text for clickable steps
                instruction_for_choice_text = "Click this if omitting this step would significantly change the target step (in yellow)."
                html += f'{instruction_for_choice_text}'
            html += '</div>'
            
            # Right column: step box with consistent width
            html += '<div class="step-box">'
            
            if step_idx == target_idx:
                # if the current quesiton is an attention check, change the target step to have a text that says to # select the crucial step
                if i in attention_check_indices:
                    step_text = f"This is an attention check. Select option {q['correct_answers'][0]}."
                # This is the target step - highlight it
                html += f"""
            <div class="step target-step">
                <strong>Step {step_idx} (Target Step)</strong>
                <p>{step_text}</p>
            </div>
"""
            elif step_idx in choice_letters:
                # This is a choice step - make it clickable
                letter = choice_letters[step_idx]
                html += f"""
            <div class="step choice" id="step-q{i+1}-{letter}" data-letter="{letter}" onclick="selectChoice('q{i+1}-{letter}')">
                <strong>Step {step_idx} <span class="choice-indicator">(Option {letter})</span></strong>
                <p>{step_text}</p>
                <input type="checkbox" name="q{i+1}" value="{letter}" style="display:none;">
            </div>"""
            else:
                # This is a regular step - not clickable
                html += f"""
            <div class="step">
                <strong>Step {step_idx}</strong>
                <p>{step_text}</p>
            </div>
"""
            
            html += '</div>'  # End of step-box div
            html += '</div>'  # End of step-container div
            
            
            
            
            # If THIS was the target step, add the counterfactual text div now (if toggle is True)
            if step_idx == target_idx and counterfactual_text and show_alt_box_toggle:
                new_preface = "If the crucial step (one of the options above) was omitted, the AI will write instead:" # Shorter preface
                # "If the crucial step (one of the options above) was omitted, the text the AI write here would bebegin with:"
                display_counterfactual_text = f"{counterfactual_text}..."
                html += f"""
        <div class="step-container">
            <div class="instruction-area">
                {new_preface}
            </div>
            <div class="step-box">
                <div class="counterfactual-text-info">
                    <strong style="font-style: normal;">Alternative step</strong><br>
                    <p style="font-style: normal;">{display_counterfactual_text}</p>
                </div>
            </div>
        </div>
"""
        
        # Add results area and navigation controls
        prev_disabled = "" # Will be managed by JavaScript
        next_disabled = "" # Will be managed by JavaScript
        
        html += f"""
            </div>
            
            <div class="result hidden" id="result-{i+1}"></div>
            
            <div class="controls">
            
"""
        # Only include Check Answer and Show Correct Answer buttons if not in prolific mode
        if not prolific_mode:
            html += f"""                <button onclick="checkAnswer({i+1}, [{correct_answers_json}])" id="check-btn-{i+1}">Check Answer</button>
                <button onclick="showCorrectAnswer({i+1}, [{correct_answers_json}])" id="show-ans-btn-{i+1}">Show Correct Answer</button>
"""
        html += f"""                <button onclick="nextQuestion({i+1})" id="next-btn-{i+1}" {next_disabled}>Next</button>
            </div>
        </div>
"""
    
    # Add Submit All button and status div after all questions
    html += """
        <div class="submit-container">
            <button onclick="submitAllAnswers()" id="submit-all-btn">Submit All Answers</button>
            <div id="submit-status"></div>
        </div>
    </div> <!-- End of test-container -->
"""
    
    # Add JavaScript for interactivity
    html += """
    <script>
        // Define prolific_mode in JavaScript scope
        const prolific_mode = """ + str(prolific_mode).lower() + """;
        
        // Store user answers and demographic information
        const userAnswers = {};
        const demographicInfo = {}; // Ensure this is initialized globally in the script
        
        // Timer variables
        let testStartTime = 0;
        let currentQuestionStartTime = 0;
        let timeSpentPerQuestion = {};
        let totalTimeSpent = 0;
        
        // Question order management
        let shuffledQuestionOrder = [];
        let currentQuestionIndex = 0;

        function handleEducationChange() {
            const educationSelect = document.getElementById('education');
            const bachelorDegreeContainer = document.getElementById('bachelor-degree-field-container');
            const bachelorDegreeInput = document.getElementById('bachelor-degree-field');

            const educationValue = educationSelect.value;
            // Show Bachelor's degree field if education is Bachelor's or higher, or some college/associates
            const relevantDegreesForBachelorField = ['bachelors', 'masters', 'doctorate'];

            if (relevantDegreesForBachelorField.includes(educationValue)) {
                bachelorDegreeContainer.classList.remove('hidden');
            } else {
                bachelorDegreeContainer.classList.add('hidden');
                if(bachelorDegreeInput) bachelorDegreeInput.value = ''; // Clear if hidden
            }
        }
        
        // Function to shuffle array
        function shuffleArray(array) {
            const shuffled = [...array];
            for (let i = shuffled.length - 1; i > 0; i--) {
                const j = Math.floor(Math.random() * (i + 1));
                [shuffled[i], shuffled[j]] = [shuffled[j], shuffled[i]];
            }
            return shuffled;
        }
        
        // Function to create shuffled question order
        function createShuffledQuestionOrder() {
            // Get all question containers
            const questionContainers = document.querySelectorAll('.question-container');
            
            // Group questions by sample and model
            const groups = {};
            questionContainers.forEach(container => {
                const sampleIdx = container.getAttribute('data-sample-idx');
                const modelName = container.getAttribute('data-model-name');
                const groupKey = `${sampleIdx}-${modelName}`;
                
                if (!groups[groupKey]) {
                    groups[groupKey] = [];
                }
                
                groups[groupKey].push({
                    originalIndex: parseInt(container.getAttribute('data-original-index')),
                    sampleIdx: parseInt(sampleIdx),
                    modelName: modelName
                });
            });
            
            // Sort questions within each group by original index to maintain order
            Object.keys(groups).forEach(groupKey => {
                groups[groupKey].sort((a, b) => a.originalIndex - b.originalIndex);
            });
            
            // Convert groups to array and shuffle the groups
            const groupsArray = Object.values(groups);
            const shuffledGroups = shuffleArray(groupsArray);
            
            // Flatten the shuffled groups to get the final order
            const shuffledOrder = [];
            shuffledGroups.forEach(group => {
                group.forEach(question => {
                    shuffledOrder.push(question.originalIndex);
                });
            });
            
            return shuffledOrder;
        }
        
        // Function to show question by shuffled index
        function showQuestionByShuffledIndex(shuffledIndex) {
            // Hide all questions
            document.querySelectorAll('.question-container').forEach(container => {
                container.style.display = 'none';
            });
            
            // Show the question at the shuffled index
            if (shuffledIndex >= 0 && shuffledIndex < shuffledQuestionOrder.length) {
                const originalQuestionNumber = shuffledQuestionOrder[shuffledIndex];
                const questionContainer = document.getElementById(`question-container-${originalQuestionNumber}`);
                if (questionContainer) {
                    questionContainer.style.display = 'block';
                }
                
                // Update progress counter
                document.getElementById('current-question').textContent = shuffledIndex + 1;
                
                // Update navigation buttons
                updateNavigationButtons(shuffledIndex);
            }
        }
        
        // Function to update navigation button states
        function updateNavigationButtons(currentIndex) {
            const totalQuestions = shuffledQuestionOrder.length;
            
            // Update all previous buttons
            document.querySelectorAll('[id^="prev-btn-"]').forEach(btn => {
                btn.disabled = currentIndex === 0;
            });
            
            // Update all next buttons
            document.querySelectorAll('[id^="next-btn-"]').forEach(btn => {
                btn.disabled = currentIndex === totalQuestions - 1;
            });
        }

        // Function to initialize question order
        function initializeQuestionOrder() {
            shuffledQuestionOrder = createShuffledQuestionOrder();
            currentQuestionIndex = 0;
            
            console.log('Shuffled question order:', shuffledQuestionOrder); // Client-side debug
            
            // Show the first question
            showQuestionByShuffledIndex(0);
        }
        
        // Function to start the test (hide demographics, show test)
        function startTest() {
            // This block is for when the demographic form is visible and its "Start Test" button is clicked
            if (prolific_mode && document.getElementById('demographic-container') && document.getElementById('demographic-container').style.display !== 'none') {
                console.log("Attempting to collect demographic info..."); // Client-side debug
                const missing = [];
                const profID = document.getElementById('prolific-id').value.trim();
                const education = document.getElementById('education').value;
                const age = document.getElementById('age').value;
                const pronouns = document.getElementById('pronouns').value;
                const freq = document.getElementById('frequency').value;
                const familiar = document.querySelector('input[name="familiar-feature"]:checked');
                const likert1 = document.querySelector('input[name="likert1"]:checked');
                const likert2 = document.querySelector('input[name="likert2"]:checked');
                const likert3 = document.querySelector('input[name="likert3"]:checked');
                const bachelorField = document.getElementById('bachelor-degree-field').value.trim();
                
                // Validation
                if (!profID) missing.push('Prolific ID');
                if (!education) missing.push('Education level');
                if (!age) missing.push('Age');
                if (!pronouns) missing.push('Preferred pronouns');
                if (!freq) missing.push('AI use frequency');
                if (!familiar) missing.push('Feature familiarity');
                if (!likert1) missing.push('Likert scale 1 (transparency)');
                if (!likert2) missing.push('Likert scale 2 (trust)');
                if (!likert3) missing.push('Likert scale 3 (performance)');
                
                if (missing.length > 0) {
                    alert('Please complete the following demographic fields: ' + missing.join(', '));
                    console.error('Demographic validation failed:', missing); // Client-side debug
                    return; // Exit if validation fails, demographicInfo remains {}
                }
                
                // Populate demographicInfo object
                demographicInfo.prolificId = profID;
                demographicInfo.education = education;
                demographicInfo.bachelorDegreeField = bachelorField || 'N/A'; // Default if empty
                demographicInfo.age = age;
                demographicInfo.pronouns = pronouns;
                demographicInfo.frequency = freq;
                demographicInfo.familiarFeature = familiar.value; // .value is safe due to validation
                demographicInfo.likert1 = likert1.value;
                demographicInfo.likert2 = likert2.value;
                demographicInfo.likert3 = likert3.value;
                
                console.log('Demographic info collected:', JSON.stringify(demographicInfo)); // CRITICAL: Check this in browser console
                
                document.getElementById('demographic-container').style.display = 'none';
                const testContainer = document.getElementById('test-container');
                testContainer.style.display = 'block';
                testContainer.classList.remove('hidden');
                
                testStartTime = Date.now(); // Start timer after demographics
                currentQuestionStartTime = testStartTime;
                initializeQuestionOrder(); // Setup and show the first actual test question
                window.scrollTo(0, 0);
                return; // Important to prevent fall-through if any other logic existed below
            }

            // Fallback or non-prolific mode logic (demographicInfo will remain {} if this path is taken)
            if (!prolific_mode) {
                console.log("Prolific mode is off, skipping demographic collection via form.");
                document.getElementById('test-container').style.display = 'block';
                document.getElementById('test-container').classList.remove('hidden');
                if (document.getElementById('demographic-container')) {
                    document.getElementById('demographic-container').style.display = 'none';
                }
                testStartTime = Date.now();
                currentQuestionStartTime = testStartTime;
                initializeQuestionOrder();
                window.scrollTo(0, 0);
            }
        }
        
        // Function to select/deselect a choice
        function selectChoice(choiceId) {
            // Get information from the step element
            const stepElement = document.getElementById(`step-${choiceId}`);
            const questionNum = choiceId.split('-')[0].substring(1);
            const choiceLetter = choiceId.split('-')[1];
            
            // For single choice, deselect other selected choices for this question
            document.querySelectorAll(`[id^="step-q${questionNum}-"]`).forEach(siblingStep => {
                if (siblingStep.id !== `step-${choiceId}` && siblingStep.classList.contains('selected')) {
                    siblingStep.classList.remove('selected');
                    const siblingCheckbox = siblingStep.querySelector('input[type="checkbox"]');
                    if (siblingCheckbox) siblingCheckbox.checked = false;
                }
            });
            
            stepElement.classList.toggle('selected');
            const checkbox = stepElement.querySelector('input[type="checkbox"]');
            checkbox.checked = stepElement.classList.contains('selected');
            
            // Save answer (now single choice)
            if (checkbox.checked) {
                userAnswers[questionNum] = [choiceLetter]; // Store as an array with one item
            } else {
                delete userAnswers[questionNum]; // Or userAnswers[questionNum] = [];
            }
        }
        
        // Function to check the answer
        function checkAnswer(questionNum, correctAnswers) {
            const resultDiv = document.getElementById(`result-${questionNum}`);
            const selectedAnswers = userAnswers[questionNum] || []; // Should be an array with 0 or 1 item
            
            // Sort both arrays to compare them regardless of order (correctAnswers is already sorted by generation)
            const sortedSelected = [...selectedAnswers].sort();
            const sortedCorrect = [...correctAnswers].sort(); // correctAnswers is an array of single correct letter e.g. ['B']
            
            // Check if answers match
            const isCorrect = sortedSelected.length === sortedCorrect.length && 
                              sortedSelected.every((value, index) => value === sortedCorrect[index]);
            
            if (isCorrect) {
                resultDiv.textContent = 'Correct! You identified the crucial step.';
                resultDiv.className = 'result correct';
                resultDiv.style.backgroundColor = '#d4edda';
                resultDiv.style.color = '#155724';
            } else {
                resultDiv.textContent = `Incorrect. The correct answer is ${sortedCorrect.join('')}. Try again or click \\"Show Correct Answer\\".`;
                resultDiv.className = 'result incorrect';
                resultDiv.style.backgroundColor = '#f8d7da';
                resultDiv.style.color = '#721c24';
            }
            
            resultDiv.classList.remove('hidden');
            
            // Update choices to show correct/incorrect
            document.querySelectorAll(`[id^="step-q${questionNum}-"]`).forEach(step => {
                if (!step.classList.contains('choice')) return;
                
                const choiceLetter = step.getAttribute('data-letter');
                const isSelected = step.classList.contains('selected');
                const shouldBeSelected = correctAnswers.includes(choiceLetter);
                
                step.classList.remove('correct', 'incorrect');
                if (isSelected && shouldBeSelected) {
                    step.classList.add('correct');
                } else if (isSelected && !shouldBeSelected) {
                    step.classList.add('incorrect');
                } else if (!isSelected && shouldBeSelected) { // Also mark the correct one if not selected by user
                    // step.classList.add('correct'); // Optionally highlight the correct one even if not picked by user during check
                }
            });
        }
        
        // Function to show the correct answer
        function showCorrectAnswer(questionNum, correctAnswers) { // correctAnswers is an array like ['B']
            // Reset all selections and badges
            document.querySelectorAll(`[id^="step-q${questionNum}-"]`).forEach(step => {
                if (!step.classList.contains('choice')) return;
                
                const choiceLetter = step.getAttribute('data-letter');
                const checkbox = step.querySelector('input[type="checkbox"]');
                const strongTag = step.querySelector('strong');
                const existingBadge = strongTag ? strongTag.querySelector('.badge') : null;
                if (existingBadge) {
                    existingBadge.remove();
                }
                
                step.classList.remove('selected', 'correct', 'incorrect');
                checkbox.checked = false;
                
                // Select only correct answers
                if (correctAnswers.includes(choiceLetter)) {
                    step.classList.add('selected', 'correct');
                    checkbox.checked = true;
                    
                    // Add a crucial badge
                    if (strongTag && !strongTag.querySelector('.badge-crucial')) {
                        const badge = document.createElement('span');
                        badge.className = 'badge badge-crucial';
                        badge.textContent = 'Crucial';
                        strongTag.appendChild(badge);
                    }
                } else {
                    // Add a not-crucial badge
                    if (strongTag && !strongTag.querySelector('.badge-not-crucial')) {
                        const badge = document.createElement('span');
                        badge.className = 'badge badge-not-crucial';
                        badge.textContent = 'Not Crucial';
                        strongTag.appendChild(badge);
                    }
                }
            });
            
            // Update user answers to reflect the correct one shown
            userAnswers[questionNum] = [...correctAnswers];
            
            // Show result
            const resultDiv = document.getElementById(`result-${questionNum}`);
            resultDiv.textContent = `The crucial step is: ${correctAnswers.join(', ')}.`;
            resultDiv.className = 'result correct';
            resultDiv.style.backgroundColor = '#d4edda';
            resultDiv.style.color = '#155724';
            resultDiv.classList.remove('hidden');
        }
        
        // Record time spent on current question and update total time
        function recordQuestionTime(originalQuestionNum) {
            const now = Date.now();
            const timeSpentInSeconds = (now - currentQuestionStartTime) / 1000;
            timeSpentPerQuestion[originalQuestionNum] = timeSpentInSeconds;
            currentQuestionStartTime = now; // Reset for next question
            
            // Also update total time
            totalTimeSpent = (now - testStartTime) / 1000;
            
            console.log(`Time spent on question ${originalQuestionNum}: ${timeSpentInSeconds.toFixed(2)}s, Total: ${totalTimeSpent.toFixed(2)}s`);
        }
        
        // Function to navigate to the next question
        function nextQuestion(currentOriginalNum) {
            // Record time spent on this question using original question number
            recordQuestionTime(currentOriginalNum);
            // if the current question was not answered, do not move to next question
            if (!userAnswers[currentOriginalNum] || userAnswers[currentOriginalNum].length === 0) {
                const submitStatusDiv = document.getElementById('submit-status');
                submitStatusDiv.textContent = 'Please answer the current question before moving to the next one.';
                submitStatusDiv.style.color = 'red';
                return; // Do not proceed if current question is unanswered
            }
            // Move to next question in shuffled order
            if (currentQuestionIndex < shuffledQuestionOrder.length - 1) {
                // Clear any error messages when successfully navigating
                const submitStatusDiv = document.getElementById('submit-status');
                submitStatusDiv.textContent = '';
                
                currentQuestionIndex++;
                showQuestionByShuffledIndex(currentQuestionIndex);
                window.scrollTo(0, 0);
                
                // In Prolific mode, send the current answers when moving to next question
                if (prolific_mode) {
                    submitCurrentProgress(false);
                }
            }
        }
        
        // Function to navigate to the previous question
        function prevQuestion(currentOriginalNum) {
            // Record time spent on this question using original question number
            recordQuestionTime(currentOriginalNum);
            
            // Move to previous question in shuffled order
            if (currentQuestionIndex > 0) {
                currentQuestionIndex--;
                showQuestionByShuffledIndex(currentQuestionIndex);
                window.scrollTo(0, 0);
            }
        }
        
        // Function to handle successful submission
        function onSubmissionSuccess(response) {
            const submitStatusDiv = document.getElementById('submit-status');
            const submitButton = document.getElementById('submit-all-btn');
            if (response.result === 'success') {
                let successMessage = 'Submission successful! Thank you. Message: ' + response.message;
                submitStatusDiv.innerHTML = successMessage; // Use innerHTML to render the strong tag
                submitStatusDiv.style.color = 'green';
                document.querySelectorAll('.controls button').forEach(btn => btn.disabled = true);
                document.querySelectorAll('.step.choice').forEach(step => step.onclick = null);
            } else {
                submitStatusDiv.textContent = 'Submission failed. Please try again. Error: ' + (response.message || 'Unknown error');
                submitStatusDiv.style.color = 'red';
                submitButton.disabled = false;
            }
        }
        
        // Function to handle submission failure
        function onSubmissionFailure(error) {
            const submitStatusDiv = document.getElementById('submit-status');
            const submitButton = document.getElementById('submit-all-btn');
            console.error('Error submitting answers:', error);
            submitStatusDiv.textContent = 'Submission failed due to a script error. Details: ' + error.message;
            submitStatusDiv.style.color = 'red';
            submitButton.disabled = false;
        }
        
        // Function to submit current progress
        function submitCurrentProgress(isFinal) {
            // If this is the final submission, record time for the last question
            if (isFinal) {
                const currentOriginalQuestionNum = shuffledQuestionOrder[currentQuestionIndex];
                recordQuestionTime(currentOriginalQuestionNum);
            }
            
            const submissionData = {
                timestamp: new Date().toISOString(),
                answers: userAnswers,
                demographic: demographicInfo, // This sends the (potentially empty) demographicInfo
                testTitle: document.title,
                totalQuestions: shuffledQuestionOrder.length, // Use shuffled length
                isComplete: isFinal,
                timePerQuestion: timeSpentPerQuestion,
                totalTimeSeconds: totalTimeSpent,
                shuffledQuestionOrder: shuffledQuestionOrder
            };
            
            console.log("Submitting data:", JSON.stringify(submissionData)); // Client-side debug
            
            if (isFinal) {
                // Final submission uses the regular handlers
                google.script.run
                    .withSuccessHandler(onSubmissionSuccess)
                    .withFailureHandler(onSubmissionFailure)
                    .saveAnswers(submissionData);
            } else {
                // Progress submissions use inline anonymous functions instead of separate named handlers
                // This fixes the "onProgressSubmissionFailure is not defined" error
                google.script.run
                    .withSuccessHandler(function(response) {
                        console.log('Progress saved successfully:', response);
                        // No UI feedback for progress saves to avoid disrupting the user experience
                    })
                    .withFailureHandler(function(error) {
                        console.error('Error submitting progress:', error);
                        // No UI feedback for failures to avoid disrupting the user experience
                    })
                    .saveAnswers(submissionData);
            }
        }

        // Function to submit all answers
        function submitAllAnswers() {
            const submitStatusDiv = document.getElementById('submit-status');
            const submitButton = document.getElementById('submit-all-btn');
            
            // Check if there are any answers at all
            if (Object.keys(userAnswers).length === 0) {
                submitStatusDiv.textContent = 'Please answer at least one question before submitting.';
                submitStatusDiv.style.color = 'red';
                return;
            }
            
            // Check for unanswered questions - use original question numbers
            const totalQuestions = shuffledQuestionOrder.length;
            const missingQuestions = [];
            
            for (let i = 0; i < totalQuestions; i++) {
                const originalQuestionNum = shuffledQuestionOrder[i];
                if (!userAnswers[originalQuestionNum] || userAnswers[originalQuestionNum].length === 0) {
                    missingQuestions.push(i + 1); // Display position (1-indexed)
                }
            }
            
            // If there are unanswered questions, show an error
            if (missingQuestions.length > 0) {
                submitStatusDiv.textContent = `Please answer all questions before submitting. Missing answers for question${missingQuestions.length > 1 ? 's' : ''}: ${missingQuestions.join(', ')}`;
                submitStatusDiv.style.color = 'red';
                
                // Optionally, scroll to the first unanswered question
                const firstMissingDisplayPosition = missingQuestions[0];
                const firstMissingShuffledIndex = firstMissingDisplayPosition - 1;
                
                if (currentQuestionIndex !== firstMissingShuffledIndex) {
                    currentQuestionIndex = firstMissingShuffledIndex;
                    showQuestionByShuffledIndex(currentQuestionIndex);
                    window.scrollTo(0, 0);
                    submitStatusDiv.textContent += ' - Taking you to the first unanswered question.';
                }
                
                return;
            }
            
            submitButton.disabled = true;
            submitStatusDiv.textContent = 'Submitting...';
            submitStatusDiv.style.color = 'blue';
            
            // In Prolific mode, use the common submission function with final=true
            submitCurrentProgress(true);
        }

        // Add event listener for DOMContentLoaded to initialize conditional question visibility
        document.addEventListener('DOMContentLoaded', function() {
            if (document.getElementById('education')) { 
                 handleEducationChange(); 
            }
            
            // If not in prolific mode, test starts immediately.
            // If in prolific mode, startTest() is called by the button on the demographic form.
            // initializeQuestionOrder() will be called by startTest().
            if (!prolific_mode) {
                startTest(); // Directly start if not prolific mode
            }
        });
    </script>
</body>
</html>"""

    return html

<>:150: SyntaxWarning: invalid escape sequence '\H'
<>:150: SyntaxWarning: invalid escape sequence '\H'
/var/folders/qw/23_w72c96k56zj7rpll5_hhh0000gn/T/ipykernel_86639/3971378709.py:150: SyntaxWarning: invalid escape sequence '\H'
  <option value="he">He\Him</option>


In [27]:
formatted_test

{'title': 'Step Dependency Questionnaire',
 'instructions': "You are presented with a problem and a chain of thought generated by an AI when solving it. For each question, the chain of thought is shown up to a 'target step' (highlighted in yellow).\n\n<strong>Your task:</strong> From the four options provided, select the step that is crucial for the target step. All other steps being equal, the crucial step absence would cause the AI to generate a significantly different target step. \n\nOnly one of the four options is crucial. The other three, if absent, would not significantly change the target step.\n\nA snippet of text below the target step shows how the target step in yellow will begin if the crucial step was indeed omitted. Use this as a hint.",
 'samples': {'deepseek-ai/DeepSeek-R1': {0: {'sample_idx': 0,
    'links': [{'omitted_step_index': 1,
      'target_step_index': 4,
      'original_step': ' It must be 150% of the original value before repairs, right?',
      'alternative

In [28]:
# ==============================================================================
# GENERATE AND SAVE HTML TEST INTERFACE
# ==============================================================================

# Generate complete HTML with configured options
html_test_content = generate_html_test(
    formatted_test, 
    show_alternative_step_box, 
    prolific_mode
)

# Save HTML file to test directory
test_file_path = os.path.join(directory_name, 'test.html')
with open(test_file_path, 'w', encoding='utf-8') as f:
    f.write(html_test_content)

print(f"\nHTML test interface saved to: {test_file_path}")
print(f"  - Alternative step hints: {'Enabled' if show_alternative_step_box else 'Disabled'}")
print(f"  - Prolific mode: {'Enabled' if prolific_mode else 'Disabled'}")




HTML test interface saved to: tests/deepseek-ai_DeepSeek-R1_gsm8k_1_samples_20251117-172503/test.html
  - Alternative step hints: Enabled
  - Prolific mode: Enabled


In [29]:
# ==============================================================================
# TEST GENERATION COMPLETE
# ==============================================================================

print(f"\n{'='*70}")
print(f"Test Generation Complete")
print(f"{'='*70}")
print(f"\nOutput directory: {directory_name}")
print(f"\nGenerated files:")
print(f"  1. test_questions.json - Raw question data")
print(f"  2. parent_step_detection_test.json - Formatted test structure")
print(f"  3. formatted_test.json - Additional formatted copy")
print(f"  4. test.html - Interactive web interface")
print(f"\nThe test is ready for administration.")


Test Generation Complete

Output directory: tests/deepseek-ai_DeepSeek-R1_gsm8k_1_samples_20251117-172503

Generated files:
  1. test_questions.json - Raw question data
  2. parent_step_detection_test.json - Formatted test structure
  3. formatted_test.json - Additional formatted copy
  4. test.html - Interactive web interface

The test is ready for administration.


In [30]:
# Display formatted test questions for review (optional)
formatted_test['questions']

[{'question_number': 1,
  'sample_idx': 0,
  'main_question': 'Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?',
  'target_step_index': 9,
  'target_step_text': '\n\nLet me calculate that.',
  'counterfactual_starting_text': ' That means the value went up by',
  'choices': [{'choice_letter': 'A',
    'step_index': 3,
    'text': ' The increase is 150%, but 150% of what?'},
   {'choice_letter': 'B',
    'step_index': 4,
    'text': ' It must be 150% of the original value before repairs, right?'},
   {'choice_letter': 'C',
    'step_index': 5,
    'text': ' Because the repairs increased the value based on the original.'},
   {'choice_letter': 'D',
    'step_index': 7,
    'text': ' Then he put in $50,000 for repairs, and that increased the value by 150% of the original value.'}],
  'correct_answers': ['A'],
  'model_name': 'deepseek-ai/DeepSeek-R1'},
 {'qu